In [ ]:
from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c superai5-esan-to-thai-machine-translation
!unzip /content/superai5-esan-to-thai-machine-translation.zip -d machine_translation

  0% 0.00/72.1k [00:00<?, ?B/s]
100% 72.1k/72.1k [00:00<00:00, 47.1MB/s]
Archive:  /content/superai5-esan-to-thai-machine-translation.zip
  inflating: machine_translation/sample_submission.csv  
  inflating: machine_translation/submission.csv  
  inflating: machine_translation/test.csv  
  inflating: machine_translation/train.csv  
  inflating: machine_translation/val.csv  


In [ ]:
!pip install pythainlp transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

import torch
import sentencepiece as spm
from pythainlp.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('/content/machine_translation/train.csv', index_col='id')
val_df = pd.read_csv('/content/machine_translation/val.csv', index_col='id')
df = pd.concat([train_df, val_df], axis=0)
df.head()

,input,output
id,,
0,คึดฮอดเจ้าหลายเด้อ\n,คิดถึงเธอมากนะ\n
1,เจ้าสิไปเบิ่งคอนเสิร์ตหมอลำซิ่งนำกันบ่มื้อนี้แ...,คุณจะไปดูคอนเสิร์ตหมอลำซิ่งด้วยกันไหมเย็นนี้\n
2,เฮาสิไปเลาะในเมือง\n,เราจะไปเดินเล่นในเมือง\n
3,เฮามาเฮ็ดแนวกินนำกัน\n,เรามาทำอาหารด้วยกัน\n
4,เมื่อวานเฮาไปฟังหมอลำหน้าวัดม่วนหลายสนุกจนบ่มี...,เมื่อวานเราไปฟังหมอลำหน้าวัดสนุกมากจนไม่มีใครอ...


## Preprocessing

In [ ]:
def clean_text(text):
    """Remove extra spaces and special characters."""
    text = text.strip()  # Remove leading/trailing spaces
    text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)  # Keep only Thai characters and spaces
    text = re.sub(r'\s+', ' ', text)  # Normalize multiple spaces
    return text

def tokenize_text(text):
    return ' '.join(word_tokenize(text, engine='newmm'))

def preprocess_dataset(df):
    df['input'] = df['input'].apply(clean_text)
    df['output'] = df['output'].apply(clean_text)

    df['input'] = df['input'].apply(tokenize_text)
    df['output'] = df['output'].apply(tokenize_text)

    return df

df = preprocess_dataset(df)
df.head()

,input,output
id,,
0,คึด ฮอด เจ้า หลาย เด้อ,คิดถึง เธอ มาก นะ
1,เจ้า สิ ไป เบิ่ง คอนเสิร์ต หมอลำ ซิ่ง นำ กัน บ...,คุณ จะ ไปดู คอนเสิร์ต หมอลำ ซิ่ง ด้วยกัน ไหม เ...
2,เฮา สิ ไป เลาะ ใน เมือง,เรา จะ ไป เดินเล่น ใน เมือง
3,เฮา มา เฮ็ด แนว กิน นำ กัน,เรา มา ทำอาหาร ด้วยกัน
4,เมื่อวาน เฮา ไป ฟัง หมอลำ หน้า วัด ม่วน หลาย ส...,เมื่อวาน เรา ไป ฟัง หมอลำ หน้า วัด สนุก มาก จน...


In [ ]:
esan_texts = df["input"].tolist()
thai_texts = df["output"].tolist()

with open("esan_train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(esan_texts))

with open("thai_train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(thai_texts))

# Train SentencePiece Tokenizer (shared vocab for ESAN & Thai)
spm.SentencePieceTrainer.train(
    input="esan_train.txt,thai_train.txt",
    model_prefix="esan_thai_spm",
    vocab_size=1500,  # Adjust based on dataset size
    character_coverage=0.9995,
    model_type="unigram"
)

# Load Trained SentencePiece Model
sp = spm.SentencePieceProcessor(model_file="esan_thai_spm.model")

In [ ]:
df["input"] = df["input"].apply(lambda x: " ".join(sp.encode(x, out_type=str)))
df["output"] = df["output"].apply(lambda x: " ".join(sp.encode(x, out_type=str)))

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.head()

,input,output
id,,
818,▁ไป ▁ตลาด ▁มา ▁ได้ ▁หยัง,▁ไป ▁ตลาด ▁มา ▁ได้ ▁อะไร
887,▁มี ▁บริการ ▁ตรวจ ▁สุขภาพ ▁ถึง ▁บ้าน ▁บ่,▁มี ▁บริการ ▁ตรวจ ▁สุขภาพ ▁ถึง ▁บ้าน ▁ไหม
788,▁ใจ เย็น ▁ๆ ▁เด้อ,▁ใจ เย็น ▁ๆ ▁นะ
1530,▁เจ้า ▁มี ▁เวลา ▁บ่,▁คุณ ▁มี ▁เวลา ▁หรือเปล่า
619,▁รถ บ ัส ▁เสียห ลัก ▁ตก ▁ถ นน ▁ บาด เจ็บ ▁หลาย...,▁รถ บ ัส ▁เสียห ลัก ▁ตก ▁ถ นน ▁มี ▁ผู้ บาด เจ็...


## Model Training

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

model_name = "facebook/mbart-large-50"
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="th_TH", tgt_lang="th_TH")
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Tokenize Function
def preprocess_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]

    return inputs

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenize Data
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Map:   0%|          | 0/1644 [00:00<?, ? examples/s]

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

In [ ]:
# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    run_name="esan-to-thai-experiment",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    disable_tqdm=False,
    report_to="none",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


KeyboardInterrupt: 

## Prediction

In [ ]:
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=128, num_beams=5)
    decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return decoded_text


val_df["input"].apply(translate_text)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import sacrebleu

# Convert to List Format for sacrebleu
references = [ref_df["output"].tolist()]  # BLEU expects list of reference lists
hypotheses = test_df["predicted_output"].tolist()

# Compute BLEU Score
bleu = sacrebleu.corpus_bleu(hypotheses, references)
print(f"BLEU Score: {bleu.score:.2f}")

# Compute chrF++ Score
chrf = sacrebleu.corpus_chrf(hypotheses, references)
print(f"chrF++ Score: {chrf.score:.2f}")

print("✅ Evaluation complete!")
